<a href="https://colab.research.google.com/github/jeffreygalle/MAT422/blob/main/MAT422_Project_Code_Appendix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programs used to clean data, preform analysis on gender, race, and regional differences for CLD Mortality.

Excel files can be found on google drive at https://drive.google.com/drive/folders/1U-eRbgiFAIlsX03zPt446qb7pSu5NBOg?usp=share_link

Under LiverDiseaseProject422/"Excel Documents"

In [ ]:
# Excel data cleaning

import pandas as pd


# Load the Excel file
file_path = '/Users/jeffgallegos/PycharmProjects/pythonProject2/ALL_StatesData_ 1.xlsx'

df_all_states = pd.read_excel(file_path)

# Strip spaces and standardize case
df_all_states['State'] = df_all_states['State'].str.strip().str.lower()

# Define the regions
regions = {
    "Region 1": ["arizona", "new mexico", "texas", "oklahoma"],
    "Region 2": ["california", "nevada", "utah", "colorado", "wyoming", "idaho", "oregon", "washington", "montana"],
    "Region 3": ["north dakota", "south dakota", "nebraska", "kansas", "minnesota", "iowa", "missouri", "wisconsin", "illinois", "michigan", "indiana", "ohio"],
    "Region 4": ["arkansas", "louisiana", "mississippi", "alabama", "tennessee", "kentucky", "florida", "georgia", "south carolina", "north carolina", "virginia", "west virginia"],
    "Region 5": ["maine", "vermont", "new hampshire", "massachusetts", "new york", "connecticut", "rhode island", "new jersey", "pennsylvania", "delaware", "district of columbia", "maryland"]
}

# Filter the data into separate regions and pivot the table
region_data = {}

for region_name, states in regions.items():
    # Filter data for the region
    filtered_data = df_all_states[df_all_states['State'].isin(states)]
    # Pivot the table so years are columns and states are rows
    pivoted_data = filtered_data.pivot(index='State', columns='Year', values='Rate').reset_index()
    region_data[region_name] = pivoted_data

# Save the pivoted data into an Excel file with separate sheets
output_path = 'LiverDiseaseRatesByRegion_Pivoted.xlsx'
with pd.ExcelWriter(output_path) as writer:
    for region_name, data in region_data.items():
        data.to_excel(writer, sheet_name=region_name, index=False)

print(f"Data saved to {output_path}")


In [ ]:
# CLD Mortality by Racial demographics

import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd


# Define the dataset
data = {
    'Year': list(range(1999, 2023)),
    'All Races': [9.4, 9.4, 9.5, 9.5, 9.5, 9.2, 9.3, 9.2, 9.7, 9.9, 10.0, 10.3, 10.8, 11.1, 11.5, 12.0, 12.5, 12.5, 12.8, 13.1, 13.5, 15.7, 17.0, 16.4],
    'Native': [18.1, 17.9, 17.2, 17.1, 17.2, 16.8, 16.8, 16.2, 18.5, 18.6, 17.2, 18.5, 18.9, 20.9, 21.2, 21.0, 23.2, 23.3, 23.6, 27.7, 27.4, 35.5, 45.6, 38.3],
    'White': [9.9, 10.0, 10.1, 10.2, 10.2, 10.0, 10.1, 10.1, 10.6, 10.8, 11.0, 11.4, 11.9, 12.4, 12.8, 13.4, 14.0, 14.1, 14.4, 14.9, 15.4, 17.9, 19.3, 18.9],
    'All Other Races': [6.6, 6.3, 6.2, 5.7, 5.7, 5.5, 5.5, 5.0, 5.3, 5.2, 5.2, 5.3, 5.5, 5.4, 5.8, 5.8, 6.0, 5.9, 6.1, 5.9, 6.1, 7.2, 7.9, 7.2]
}

# Convert DataFrame
df = pd.DataFrame(data)

# ANOVA
anova_result = f_oneway(df['All Races'], df['Native'], df['White'], df['All Other Races'])
print("ANOVA Result:")
print(f"F-statistic: {anova_result.statistic:.4f}, p-value: {anova_result.pvalue:.4e}")


# dataframe to a long format
melted_df = pd.melt(df, id_vars=['Year'], value_vars=['All Races', 'Native', 'White', 'All Other Races'],
                    var_name='Race', value_name='Death Rate')

# Tukey's HSD post hoc test
tukey_result = pairwise_tukeyhsd(endog=melted_df['Death Rate'], groups=melted_df['Race'], alpha=0.05)
print("\nTukey HSD post hoc test result:")
print(tukey_result)

# post hoc test summary
tukey_result.plot_simultaneous()
plt.title("Tukey HSD Post Hoc Test Result")
plt.xlabel("Death Rate Difference")
plt.show()

In [ ]:
# CLD Mortality by male and female


import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

data_gender = {
    'Year': list(range(1999, 2023)),
    'Female_Death_Rate': [6.4, 6.5, 6.6, 6.7, 6.5, 6.4, 6.4, 6.4, 6.5, 6.7, 6.9, 7.1, 7.6, 7.8, 7.9, 8.4, 9.0, 9.0, 9.3, 9.4, 9.8, 11.4, 12.5, 12.2],
    'Male_Death_Rate': [12.5, 12.5, 12.4, 12.3, 12.6, 12.2, 12.4, 12.2, 12.9, 13.1, 13.1, 13.7, 14.1, 14.6, 15.2, 15.7, 16.2, 16.2, 16.5, 16.9, 17.4, 20.1, 21.7, 20.8]
}

# Convert DataFrame
df_gender = pd.DataFrame(data_gender)

# Plot trends for Male and Female death rates
plt.figure(figsize=(12, 6))
plt.plot(df_gender['Year'], df_gender['Female_Death_Rate'], label="Female Death Rate", marker='o')
plt.plot(df_gender['Year'], df_gender['Male_Death_Rate'], label="Male Death Rate", marker='o')
plt.xlabel("Year")
plt.ylabel("Death Rate")
plt.title("Death Rate Trends by Gender (1999 - 2022)")
plt.legend()
plt.grid()
plt.show()

# ANOVA test
anova_gender = f_oneway(df_gender['Female_Death_Rate'], df_gender['Male_Death_Rate'])
print("ANOVA Result:")
print(f"F-statistic: {anova_gender.statistic:.4f}, p-value: {anova_gender.pvalue:.4e}")

# Tukey's HSD test
melted_df_gender = pd.melt(df_gender, id_vars=['Year'], value_vars=['Female_Death_Rate', 'Male_Death_Rate'],
                           var_name='Gender', value_name='Death Rate')

tukey_gender_result = pairwise_tukeyhsd(endog=melted_df_gender['Death Rate'], groups=melted_df_gender['Gender'], alpha=0.05)
print("\nTukey HSD Post Hoc Test Result:")
print(tukey_gender_result)

# test summary
tukey_gender_result.plot_simultaneous()
plt.title("Tukey HSD Post Hoc Test Result")
plt.xlabel("Death Rate Difference")
plt.show()


In [ ]:
# HHierarchical clustering using Ward linkage

import pandas as pd
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt

file_path = '/Users/jeffgallegos/PycharmProjects/pythonProject2/ALL_StatesDataCSV.csv'
data = pd.read_csv(file_path)

# Matrix where each row is a state and each column is a year
state_year_matrix = data.pivot(index='State', columns='Year', values='Rate')

state_year_matrix = state_year_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

# Hierarchical clustering
linked = linkage(state_year_matrix, method='ward')

# Dndrogram
plt.figure(figsize=(12, 8))
plt.gca().set_facecolor((255 / 255, 248 / 255, 231 / 255))  # Set background color to RGB (255, 248, 231)
dendrogram(linked, labels=state_year_matrix.index, orientation='top', leaf_font_size=10)
plt.title("Dendrogram of State Mortality Rates (1999-2022)")
plt.ylabel("Similarity (Distance)")
plt.xlabel("State")
plt.show()



# aiming for 3-5 clusters
cluster_labels_reduced = fcluster(linked, t=30, criterion='distance')

# Assign the cluster labels to each state
state_clusters_reduced = pd.DataFrame({'State': state_year_matrix.index, 'Cluster': cluster_labels_reduced})
state_clusters_reduced.sort_values(by='Cluster', inplace=True)


# Set y-axis range based on the data's mortality rate range
y_min, y_max = state_year_matrix.min().min(), state_year_matrix.max().max()



print(state_clusters_reduced)

# Group and print the states for each cluster
for cluster_id in sorted(state_clusters_reduced['Cluster'].unique()):
    # List states in the current cluster
    states_in_cluster = state_clusters_reduced[state_clusters_reduced['Cluster'] == cluster_id]['State'].tolist()

    # Print cluster and its states
    print(f"Cluster {cluster_id}: {', '.join(states_in_cluster)}")